In [1]:
%matplotlib inline
import sys
sys.path.append('../../trajectron')
import os
import numpy as np
np.set_printoptions(suppress=True)
import torch
import dill
import json
from helper import *
import statistics
from collections import Counter

from copy import deepcopy

from experiment_helper import *

## Load nuScenes SDK and data

In [2]:
nuScenes_devkit_path = './devkit/python-sdk/'
sys.path.append(nuScenes_devkit_path)

# Map Encoding Demo

In [3]:
file_path = '../processed/nuScenes_test_mini_full.pkl'

with open(file_path, 'rb') as f:
    eval_env = dill.load(f, encoding='latin1')
eval_scenes = eval_env.scenes


In [4]:
ph = 6
log_dir = './models'

In [5]:
# Define ROI in nuScenes Map
x_min = 773.0
x_max = 1100.0
y_min = 1231.0
y_max = 1510.0

In [6]:
layers = ['drivable_area',
          'road_segment',
          'lane',
          'ped_crossing',
          'walkway',
          'stop_line',
          'road_divider',
          'lane_divider']

In [7]:
DROPPED_FRAMES = read_dropped_frames()
mvalue = get_gt_modification()

In [8]:
model_dir = os.path.join(log_dir, 'vel_ee') 
eval_stg_vel, hyp = load_model(model_dir, eval_env, ts=12)

print(f'Number of scenes = {len(eval_scenes)}')
scenes = eval_scenes
ph = 6
shift = 0
with torch.no_grad():
    timestep = np.array([2])

    output = {}
    for sid, scene in enumerate(scenes):
        # print(f'scene-{scene.name}')
        scene_perturb = deepcopy(scene)
        for ni, n in enumerate(scene.nodes):
            if str(n.id) != 'ego':
                last_available_di = 0
                for di, _ in enumerate(n.data.data):
                    scene_perturb.nodes[ni].data.data[di][1] += mvalue
                for di, _ in enumerate(n.data.data):
                    # perturb
                    frame_idx = shift + n.first_timestep + di
                    if frame_idx in DROPPED_FRAMES:
                        scene_perturb.nodes[ni].data.data[di] = scene_perturb.nodes[ni].data.data[last_available_di]
                    else:
                        last_available_di = di
        shift = scene.timesteps
        predictions_mm = eval_stg_vel.predict(scene,
                                            timestep, ph,
                                            num_samples=1,
                                            z_mode=True, gmm_mode=True)

        predictions_mm_perturb = eval_stg_vel.predict(scene_perturb,
                                                    timestep, ph,
                                                    num_samples=1,
                                                    z_mode=True, gmm_mode=True)   

        # values will always be equal to timestep above
        pkey = list(predictions_mm_perturb.keys())[0]
        output[scene.name] = {}
        for node in predictions_mm_perturb[pkey].keys():
            output[scene.name][str(node)] = {
                'original': predictions_mm[pkey][node].tolist(),
                'perturbed': predictions_mm_perturb[pkey][node].tolist()
            }
    compute_error(output)


Loading from ./models/vel_ee/model_registrar-12.pt
Loaded!

Number of scenes = 2
0.19 0.07
